### Set autoreloading
This extension will automatically update with any changes to packages in real time

In [ ]:
%load_ext autoreload
%autoreload 2

### Import packages
We'll need the `nugraph` and `pynuml` packages imported in order to plot

In [ ]:
import nugraph as ng
import pynuml

### Configure data module
Declare a data module. Depending on where you're working, you should edit the data path below to point to a valid data location.

In [ ]:
nudata = ng.data.H5DataModule(data_path='/raid/uboone/CHEP2023/CHEP2023.gnn.h5',
                              batch_size=64)

### Configure network
In order to test a trained model, we instantiate it using a checkpoint file. These are produced during training, so if you've trained a model, there should be an associated checkpoint in your output directory that you can pass here.

In [ ]:
model = ng.models.NuGraph2.load_from_checkpoint('/raid/uboone/CHEP2023/paper.ckpt').cpu()
model.freeze()

## Configure plotting utility
Instantiate the **pynuml** utility for plotting graph objects, which will do the heavy lifting for us here!

In [ ]:
plot = pynuml.plot.GraphPlot(planes=nudata.planes,
                             classes=nudata.semantic_classes)

## Plot ground truth labels

### Iterable dataset

First we define an iterator over the test dataset:

In [ ]:
test_iter = iter(nudata.test_dataset)

### Retrieve the next graph

This block retrieves a graph from the testing dataset, and passes it through the trained model. Since we defined `test_iter` as an iterator over the dataset, the following block can be executed multiple times, and each time it's executed, it will step to the next graph in the dataset.

In [ ]:
data = next(test_iter)
model.step(data)

### Plot a single graph

We can now use pynuml's plotting utilities to plot the graph as a figure. Each time you call the above block to retrieve a new graph, you can then re-execute the plotting blocks to re-plot with the new graph.

In [ ]:
fig = plot.plot(data, target='semantic', how='true', filter='true')
fig

### Save plots to disk

We can also use plotly's `write_html` and `write_image` methods to print the figure as an interactive webpage, or in a raster graphics (ie. PNG, JPEG etc) or vector graphics (ie. PDF) format. By default this writes to a `plots` subdirectory – if you're seeing an error that this directory does not exist, simply create one, or change the path to a valid output location!

In [ ]:
md = data['metadata']
tag = f'r{md.run.item()}_sr{md.subrun.item()}_evt{md.event.item()}'
name = f'plots/{tag}_semantic_true'

fig.write_html(f'{name}.html')
fig.write_image(f'{name}.png')
fig.write_image(f'{name}.pdf')